# Underactuated Control of Linearized Systems
In the following, we will cover several basic approaches to control of underactuated systems that rely on several methods of linearization that we have already covered in the course. We will use several systems to illustrate these concepts.

## To run this notebook:

1. Clone the class demos github repo: `git clone https://github.com/Nonlinear-Control-EMAE-485/EMAE485-demos.git`
2. Ensure you are using a conda environment (or venv if you prefer) with dependencies installed 
3. cd into the repository folder
4. Launch the notebook: `jupyter notebook LQR.ipnyb`

In [2]:
import matplotlib.pyplot as plt
import mujoco
import mujoco.viewer
import numpy as np
import os
from pathlib import Path
import time
import control

First, let's look at the [acrobot](https://underactuated.mit.edu/acrobot.html#section1). Import the mujoco model. If you try to run the following cell more than once, you will get an error because mujoco does not want to redefine the model object.

In [3]:
# Configure MuJoCo to use the EGL rendering backend (requires GPU)
os.environ["MUJOCO_GL"] = "egl"

model_name = f"double_pendulum"

q_ref = [0.5, 1.0]
model_path = Path("mujoco_models") / (str(model_name) + str(".xml"))
    # Load the model and data
model = mujoco.MjModel.from_xml_path(str(model_path.absolute()))

## Balancing an underactuated pendulum
For the following exercise, we will look at balancing the acrobot in the vertical position, a classic problem in nonlinear control. The most basic way to analyze and attack this problem is with tools from linearization.
### Linearization
As we have discussed, it is always possible to linearize a nonlinear system using a Taylor expansion about some operating point. After doing so, we have the following dynamics: $x = Ax + Bu$. 
### Controllability
Before designing controllers, a reasonable question to ask is if our control goal is even possible. As it turns out, we have a general method to ask an answer this question. In general, a system is controllable if it is possible to construct an unconstrained input signal, 
$u(t)$, which will move the system from any initial state to any final state in a finite interval of time. 

As it turns out, for LTI systems we have an easy way of finding the controllability condition. If the $\mathrm{rank}(R) = n$, where $n$ is the dimension of the state variable and R is $R = [B \quad AB \quad A^2B \quad ... \quad A^{n-1}B]$, then the system is controllable (see [Wikipedia's article](https://en.wikipedia.org/wiki/Controllability#Continuous_linear_time-invariant_(LTI)_systems) on the topic for details).

For the acrobot in the vertical position:

In [4]:
def acrobot_controllability(model):
    data = mujoco.MjData(model)
    data.qpos[0] = 0.0
    data.qpos[1] = 0.0
    # parameters
    g = 9.81
    m1 = 1
    m2 = m1
    lc1 = 0.25
    lc2 = lc1
    l1 = 0.5
    l2 = l1
    
    # get M inverse
    mujoco.mj_step(model, data)
    M_inv = np.zeros((model.nv, model.nv))
    mujoco.mj_solveM(model, data, M_inv, np.eye(model.nv))

    # calculate linearized model
    dG_dq = np.array([[g * (m1 * lc1 + m2 * l1 + m2 * lc2), m2 * g * lc2],[m2 * g * lc2, m2 * g * lc2]])
    A = np.block([
        [np.zeros((2,2)), np.eye(2)],
        [M_inv @ dG_dq, np.zeros((2,2))]])
    B = np.block([
        [np.zeros((2,1))],
        [M_inv @ [[0], [1]]]])
    # use control toolbox to calculate controllability matrix
    return control.ctrb(A,B)

# output the rank
print(acrobot_controllability(model))
print(np.linalg.matrix_rank(acrobot_controllability(model)) == 4)


[[   0.           -4.26606608    0.         -112.46924591]
 [   0.           13.64757486    0.          349.75265927]
 [  -4.26606608    0.         -112.46924591    0.        ]
 [  13.64757486    0.          349.75265927    0.        ]]
True


From the above, we can see that $\mathrm{rank}(R) = 4$ for the upright position, which implies that we can achieve this control goal. But how to do it?

### Linear Quadratic Regulator (LQR)
If we linearize our dynamics around a set point, (e.g. where the pendulum is balanced in the vertical position), we can use a tool from optimal control theory called LQR. We will cover this in detail soon. Using our linearized model, we can easily write a controller that uses the Python Control Systems Library implementation of LQR to achieve the balancing task:

In [5]:
def controller(model, data):
    #put the controller here. This function is called inside the simulation.
    #pass
    g = 9.81
    m1 = 1
    m2 = m1
    lc1 = 0.25
    lc2 = lc1
    l1 = 0.5
    l2 = l1
    
    M_inv = np.zeros((model.nv, model.nv))
    mujoco.mj_solveM(model, data, M_inv, np.eye(model.nv))
    
    dG_dq = np.array([[g * (m1 * lc1 + m2 * l1 + m2 * lc2), m2 * g * lc2],[m2 * g * lc2, m2 * g * lc2]])
    A = np.block([
        [np.zeros((2,2)), np.eye(2)],
        [M_inv @ dG_dq, np.zeros((2,2))]])
    B = np.block([
        [np.zeros((2,1))],
        [M_inv @ [[0], [1]]]])
    Q = np.diag([10,10,1,1])
    R = 1
    K, S, E = control.lqr(A, B, Q, R)
    x = np.hstack((data.qpos, data.qvel))
    u = - K @ x
    data.ctrl[0] = u[0]

In [6]:
def simulate_model(model):
    
    
    data = mujoco.MjData(model)
    data.qpos[0] = 0.0
    data.qpos[1] = 0.1
    model.dof_damping[:] = 0.05

    mujoco.set_mjcb_control(controller)

    with mujoco.viewer.launch_passive(model, data) as viewer:
        viewer.cam.azimuth = 90
        viewer.cam.elevation = 5
        viewer.cam.distance =  8
        viewer.cam.lookat = np.array([0.0 , 0.0 , 0.0])
        while viewer.is_running():
            step_start = time.time()
            first_time = time.time()

            # mj_step can be replaced with code that also evaluates
            # a policy and applies a control signal before stepping the physics.
            mujoco.mj_step(model, data)
            

            # Pick up changes to the physics state, apply perturbations, update options from GUI.
            viewer.sync()

            # Rudimentary time keeping, will drift relative to wall clock.
            time_until_next_step = model.opt.timestep - (time.time() - step_start)
            if time_until_next_step > 0:
                time.sleep(time_until_next_step)

In [7]:
if __name__ == "__main__":
    # Simulate the model
    simulate_model(model)

2025-02-25 13:39:45.886 mjpython[9425:11851585] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-25 13:39:45.886 mjpython[9425:11851585] +[IMKInputSession subclass]: chose IMKInputSession_Modern


You can play around with the initial conditions and physical parameters to get an idea of this controller's robustness. As you can see, as long as we are close to the upright position, LQR does a fantastic job. 

Unfortunately, this controller won't get us up to the fixed point. For that we will need different tools.